In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
articles =pd.read_csv("/kaggle/input/h-and-m-personalized-fashion-recommendations/articles.csv")
customers_df =pd.read_csv("/kaggle/input/h-and-m-personalized-fashion-recommendations/customers.csv")
transaction_df=pd.read_csv("/kaggle/input/h-and-m-personalized-fashion-recommendations/transactions_train.csv")

In [4]:
CUSTOMER_ID_COLUMN='customer_id'
ACTIVE_COLUMN="Active"
FN_COLUMN="FN"
CLUB_MEM_STATUS_COLUMN="club_member_status"
AGE_COLUMN='age'
FSHN_NEWS_FREQ_COLUMN='fashion_news_frequency'
ZIP_COLUMN="postal_code"
AGE_GRP_COLUMN='age_group'
ARTICLE_ID_COLUMN='article_id'

Plot there is correlation betwwen the categorical columns. So we can do a correlation check and decide which columns to drop or keep. But before doing that lets  handle the missing values

In [5]:
## lets replace missing values in fashion news frequency and club member status
customers_df[CLUB_MEM_STATUS_COLUMN]=customers_df[CLUB_MEM_STATUS_COLUMN].fillna("NOT_MEMBER")
customers_df[FSHN_NEWS_FREQ_COLUMN]=customers_df[CLUB_MEM_STATUS_COLUMN].fillna("NONE")
customers_df[ACTIVE_COLUMN]=customers_df[ACTIVE_COLUMN].fillna(0)
customers_df[FN_COLUMN]=customers_df[FN_COLUMN].fillna(0)

In [6]:
# Calculate the mean of the age column
mean_age = customers_df[AGE_COLUMN].mean()

# Fill missing values in the age column with the mean
customers_df[AGE_COLUMN]=customers_df[AGE_COLUMN].fillna(mean_age)


Lets make age as a category. This with the assumption that all teens will be  instersted in similar items , all people age between 25-35 has similar intersts and so on
So the categories which we gonna consider will be:
- 0-12  if any
- 13- 25 
- 25 -35
- 35- 55
- 55 - 100
we may need to split or merge some categories in this again. But lets consider this now


In [7]:
bins = [ 13, 25, 35, 55, 100]
labels = ['13-25', '25-35', '35-55', '55-100']
customers_df[AGE_GRP_COLUMN] = pd.cut(customers_df[AGE_COLUMN], bins=bins, labels=labels, right=False)

In [8]:
customers_df.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code,age_group
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0.0,0.0,ACTIVE,ACTIVE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...,35-55
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0.0,0.0,ACTIVE,ACTIVE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...,25-35
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0.0,0.0,ACTIVE,ACTIVE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...,13-25
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0.0,0.0,ACTIVE,ACTIVE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...,35-55
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,ACTIVE,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...,35-55


In [9]:
transaction_df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [10]:
customer_selected_columns=[CUSTOMER_ID_COLUMN,AGE_GRP_COLUMN]
transaction_selected_columns=[CUSTOMER_ID_COLUMN,ARTICLE_ID_COLUMN]

merged_df=transaction_df[transaction_selected_columns].merge(customers_df[customer_selected_columns],on=[CUSTOMER_ID_COLUMN],how='inner')
aggreagated_df=merged_df.groupby([AGE_GRP_COLUMN,ARTICLE_ID_COLUMN]).agg(article_count=(ARTICLE_ID_COLUMN,"count")).reset_index().sort_values(by=[AGE_GRP_COLUMN,"article_count",],ascending=False)
aggreagated_df=aggreagated_df.reset_index(drop=True)
aggreagated_df.head()

/tmp/ipykernel_30/4237263156.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  aggreagated_df=merged_df.groupby([AGE_GRP_COLUMN,ARTICLE_ID_COLUMN]).agg(article_count=(ARTICLE_ID_COLUMN,"count")).reset_index().sort_values(by=[AGE_GRP_COLUMN,"article_count",],ascending=False)


,age_group,article_id,article_count
0,55-100,579541001,3369
1,55-100,706016001,3085
2,55-100,678942001,2964
3,55-100,399256005,2921
4,55-100,610776002,2901


In [11]:
aggreagated_df['rank'] = aggreagated_df.groupby('age_group')['article_count'].rank(ascending=False, method='first')
aggreagated_df=aggreagated_df[aggreagated_df['rank']<=12]
aggreagated_df

/tmp/ipykernel_30/3461024886.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  aggreagated_df['rank'] = aggreagated_df.groupby('age_group')['article_count'].rank(ascending=False, method='first')


,age_group,article_id,article_count,rank
0,55-100,579541001,3369,1.0
1,55-100,706016001,3085,2.0
2,55-100,678942001,2964,3.0
3,55-100,399256005,2921,4.0
4,55-100,610776002,2901,5.0
5,55-100,399256001,2789,6.0
6,55-100,673677002,2665,7.0
7,55-100,610776001,2562,8.0
8,55-100,751471001,2501,9.0
9,55-100,573716012,2376,10.0


In [22]:
results_df=customers_df[customer_selected_columns].merge(aggreagated_df,on=[AGE_GRP_COLUMN],how='left')

In [23]:
results_df[results_df[CUSTOMER_ID_COLUMN]=='00009d946eec3ea54add5ba56d5210ea898def4b46c68570cf0096d962cacc75']

,customer_id,age_group,article_id,article_count,rank
156,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,55-100,579541001,3369,1.0
157,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,55-100,706016001,3085,2.0
158,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,55-100,678942001,2964,3.0
159,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,55-100,399256005,2921,4.0
160,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,55-100,610776002,2901,5.0
161,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,55-100,399256001,2789,6.0
162,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,55-100,673677002,2665,7.0
163,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,55-100,610776001,2562,8.0
164,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,55-100,751471001,2501,9.0
165,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,55-100,573716012,2376,10.0


In [26]:
set(results_df[CUSTOMER_ID_COLUMN].unique()==customers_df[CUSTOMER_ID_COLUMN].unique())

{True}

In [19]:
submission=pd.read_csv("/kaggle/input/h-and-m-personalized-fashion-recommendations/sample_submission.csv")
submission.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0706016001 0706016002 0372860001 0610776002 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0706016001 0706016002 0372860001 0610776002 07...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0706016001 0706016002 0372860001 0610776002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0706016001 0706016002 0372860001 0610776002 07...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0706016001 0706016002 0372860001 0610776002 07...


In [20]:
submission["prediction"][0]

'0706016001 0706016002 0372860001 0610776002 0759871002 0464297007 0372860002 0610776001 0399223001 0706016003 0720125001 0156231001'

In [28]:
results_df=results_df[[CUSTOMER_ID_COLUMN,ARTICLE_ID_COLUMN]]
results_df

,customer_id,article_id
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,706016001
1,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,706016002
2,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,372860001
3,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,610776002
4,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,372860002
...,...,...
16463755,ffffd9ac14e89946416d80e791d064701994755c3ab686...,610776001
16463756,ffffd9ac14e89946416d80e791d064701994755c3ab686...,751471001
16463757,ffffd9ac14e89946416d80e791d064701994755c3ab686...,573716012
16463758,ffffd9ac14e89946416d80e791d064701994755c3ab686...,706016002


In [32]:
results_df= results_df.groupby(CUSTOMER_ID_COLUMN)[ARTICLE_ID_COLUMN].agg(lambda x: ' '.join(x.astype(str))).reset_index()
results_df

,customer_id,article_id
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,706016001 706016002 372860001 610776002 372860...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,706016001 610776002 372860001 464297007 706016...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,706016001 706016002 759871002 372860001 156231...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,706016001 706016002 372860001 610776002 372860...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,706016001 706016002 372860001 610776002 372860...
...,...,...
1371975,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,706016001 706016002 759871002 372860001 156231...
1371976,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,706016001 706016002 759871002 372860001 156231...
1371977,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,706016001 706016002 759871002 372860001 156231...
1371978,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,706016001 706016002 759871002 372860001 156231...
